# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import joblib

from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute

from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice

from azureml.widgets import RunDetails

from azureml.data.dataset_factory import TabularDatasetFactory

## Dataset

Access the data that will be used in this project. 

In [2]:
ws = Workspace.from_config()
experiment_name = 'HeartFailureClassificationExp'

experiment=Experiment(ws, experiment_name)

In [3]:
# Checking an existing compute cluster or starting one
compute_name = "ComputeCluster"
try:
    compute_target = ComputeTarget(ws, compute_name)
    print(compute_name+ " already exist.")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", min_nodes=1, max_nodes=5)
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)
compute_target.wait_for_completion(show_output=True)

print(compute_target.get_status().serialize())

ComputeCluster already exist.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 1, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-02-12T08:02:33.305000+00:00', 'errors': None, 'creationTime': '2021-02-12T07:27:41.449347+00:00', 'modifiedTime': '2021-02-12T07:27:56.758676+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 5, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [5]:
ds = TabularDatasetFactory.from_delimited_files("https://raw.githubusercontent.com/eparamasari/ML_Engineer_ND_Capstone/main/data/heart_failure_clinical_records_dataset.csv")

## Hyperdrive Configuration

Explain the model used and the reason for chosing the different hyperparameters, termination policy and config settings.

In [17]:
# Creating an early termination policy with Random parameter sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, 
                                        slack_factor=0.2, 
                                        delay_evaluation=5)

# Creating the different params that will be used during training
param_sampling = RandomParameterSampling(
     {
        '--n_estimators': (20), 
        '--min_samples_split': choice(2,4,6)
     })

# Creating an estimator and hyperdrive config
estimator = SKLearn(compute_target=compute_target,
                    source_directory='./',
                    entry_script='train.py')

hyperdrive_run_config = HyperDriveConfig(estimator=estimator, 
                                         policy=early_termination_policy,
                                         hyperparameter_sampling=param_sampling,
                                         primary_metric_name="accuracy",
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=50,
                                         max_concurrent_runs=4)

In [18]:
# Submitting the experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config, show_output=True)

In [19]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_hyperdrive_run = hyperDrive_run.get_best_run_by_primary_metric()
best_hd_run_metrics = best_hyperdrive_run.get_metrics()

print('Best Run Id: ', best_hyperdrive_run.id)
print('\n Accuracy:', best_hd_run_metrics['Accuracy'])
print('\n Regularization Strength:',best_hd_run_metrics['Regularization Strength'])
print('\n Max Iterations:',best_hd_run_metrics['Max iterations'])

In [ ]:
# Saving the best model
os.makedirs("./outputs", exist_ok=True)
joblib.dump(value=best_run.id,filename='outputs/best_hyperdrive_run_model.joblib')
print("Model has been successfully saved!")

In [ ]:
# Registering the best model
model = best_hyperdrive_run.register_model(model_name='HyperDrive_HighAccuracy', model_path='outputs/', 
                                properties={'Accuracy': best_hd_run_metrics['Accuracy'],
                                            'Regularization Strength': best_hd_run_metrics['Regularization Strength'],
                                            'Max Iterations': best_hd_run_metrics['Max iterations']})

In [ ]:
# Listing registered models to verify that the model has been saved
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Downloading the environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')

# Downloading the scoring file 
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'scoreScript.py')

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service